# PyTorch: Evaluating

https://github.com/pytorch/examples/tree/master/mnist

In [70]:
import os
from glob import glob

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

import numpy as np

In [71]:
seed = 1

lr = 0.001
momentum = 0.5

batch_size = 64
test_batch_size = 64

epochs = 1
no_cuda = False
log_interval = 100

# Model

In [72]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

# Preprocess

In [73]:
datasets.MNIST('../dataset', train=True, download=True,
               transform=transforms.Compose([
                   transforms.ToTensor(),
                   transforms.Normalize((0.1307,), (0.3081,))
               ]))

Dataset MNIST
    Number of datapoints: 60000
    Root location: ../dataset
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.1307,), std=(0.3081,))
           )

In [74]:
torch.manual_seed(seed)

use_cuda = not no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}



train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True, **kwargs)




test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=test_batch_size, shuffle=True, **kwargs)

# Optimization

In [75]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

In [76]:
params = list(model.parameters())

In [77]:
for i in range(8):
    print(params[i].size())

torch.Size([20, 1, 5, 5])
torch.Size([20])
torch.Size([50, 20, 5, 5])
torch.Size([50])
torch.Size([500, 800])
torch.Size([500])
torch.Size([10, 500])
torch.Size([10])


# Training

In [81]:
for epoch in range(1, epochs + 1):
    # Train Mode
    model.train()

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()  # backpropagation 계산하기 전에 0으로 기울기 계산
        output = model(data)
        loss = F.nll_loss(output, target)  # https://pytorch.org/docs/stable/nn.html#nll-loss
        loss.backward()  # 계산한 기울기를 
        optimizer.step()  

        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            
    #Test Mode
    model.eval() # batch norm이나 dropout 등을 train mode변환
    test_loss =0
    correct = 0

    with torch.no_grad(): #auto grad engine, 즉 backpropagation 이나 gradient 계산 등을 꺼서 memory usage 를 줄이고 속도를 높임

        for data, target in test_loader : 
            date, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target , reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim = True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item() # pred과 target이 같은지 확인

    test_loss/=len(test_loader.dataset) # 평균

    print('\nTest set : Average Loss : {:.4f}, Accuracy {}/{} ({:.0f}%)\n'.format(
        test_loss, correct,len(test_loader.dataset), 100.* correct/len(test_loader.dataset)))
    
# 학습하고 테스트, 학습하고 테스트,학습하고 테스트,학습하고 테스트,학습하고 테스트

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.197943
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.334130
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.171156
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.261563
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.129276
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.128871
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.203556
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.304650
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.136920
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.171510

Test set : Average Loss : 0.1809, Accuracy 9480/10000 (95%)



# Evaluation

- 앞에서 model.train() 모드로 변한 것처럼 평가 할 때는 model.eval()로 설정
    - Batch Normalization이나 Drop Out 같은 Layer들을 잠금

In [57]:
# Test Mode
model.eval() # batch norm 이나 dropout 등을 train mode변환

Net(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)

- autograd engine, 즉 backpropagatin이나 gradient 계산 등을 꺼서 memory usage를 줄이고 속도를 높임

In [58]:
test_loss = 0
correct =0

with torch.no_grad(): # 기울기,.backpropagatin 등등  끄기
    data, target = next(iter(test_loader))
    data, target = data.to(device), target.to(device)
    output = model(data)
    
    # Model에서 나온 결과의 Loss를 계산하여 test_loss에 더함
    test_loss += F.nll_loss(output, target, reduction='sum').item() # reduction='sum' 없으면 batch size별로 따로 계산하는데, sum을 통해 1개의 스칼라로 리턴해줌.
    
    pred = output.argmax(dim= 1, keepdim =True) # pred : output을 예측한 결과 
                                                # argmax : 제일 강한 index, 컴터가 생각하는 결과
                                                # keepdim : shape 즉 차원수 유지
    correct += pred.eq(target.view_as(pred)).sum().item() # pred와 target과 같은지 확인 
                                                          # target.view_as(pred) : 차원을 같이 해줌

In [59]:
test_loss

28.209529876708984

In [60]:
target.shape

torch.Size([64])

In [61]:
output.shape # 10이었던게 argmax를 통해 64는 그대로 shape만 변형

torch.Size([64, 10])

In [62]:
pred.shape

torch.Size([64, 1])

In [63]:
target.view_as(pred)

tensor([[8],
        [4],
        [5],
        [9],
        [6],
        [0],
        [3],
        [3],
        [6],
        [6],
        [5],
        [3],
        [0],
        [5],
        [6],
        [9],
        [6],
        [5],
        [8],
        [6],
        [6],
        [4],
        [1],
        [4],
        [2],
        [0],
        [3],
        [9],
        [1],
        [7],
        [2],
        [6],
        [6],
        [4],
        [5],
        [4],
        [2],
        [8],
        [5],
        [4],
        [0],
        [1],
        [5],
        [5],
        [1],
        [6],
        [5],
        [7],
        [7],
        [0],
        [6],
        [0],
        [7],
        [1],
        [9],
        [8],
        [8],
        [9],
        [1],
        [2],
        [1],
        [3],
        [9],
        [9]])

In [64]:
pred.eq(target.view_as(pred))

tensor([[ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [False],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [False],
        [ True],
        [False],
        [ True],
        [ True],
        [ True],
        [ True],
        [False],
        [ True],
        [ True],
        [False],
        [False],
        [False],
        [ True],
        [ True],
        [ True],
        [False],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True

In [65]:
pred.eq(target.view_as(pred)).sum().item() /64
 # 얼마나 잘 맞혔는지

0.875

In [66]:
# 앞에서 얻은 test_loss의 값을 test dataset의 갯수로 나눠 평균값을 얻음
test_loss /= len(test_loader.dataset)

# test_loss , corrext 의 log확인
print("\nTest set : Average loss : {:.4f}, Accuracy : {}/{} ({:.0f}%)\n".format(
        test_loss, correct,len(test_loader.dataset),100. * correct/len(test_loader.dataset)))



Test set : Average loss : 0.0028, Accuracy : 56/10000 (1%)



In [67]:
#Test Mode
model.eval() # batch norm이나 dropout 등을 train mode변환
test_loss =0
corrext = 0

with torch.no_grad(): #auto grad engine, 즉 backpropagation 이나 gradient 계산 등을 꺼서 memory usage 를 줄이고 속도를 높임
    
    for data, target in test_loader : 
        date, target = data.to(device), target.to(device)
        ouput = model(data)
        test_loss += F.nll_loss(output, target , reduction='sum').item() # sum up batch loss
        pred = ouput.argmax(dim=1, keepdim = True) # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item() # pred과 target이 같은지 확인
    
test_loss/=len(test_loader.dataset) # 평균

print('\nTest set : Average Loss : {:.4f}, Accuracy {}/{} ({:.0f}%)\n'.format(
    test_loss, correct,len(test_loader.dataset), 100.* correct/len(test_loader.dataset)))
    




ValueError: Expected input batch_size (64) to match target batch_size (16).